# PyBullet

This is a series of notebooks related to PyBullet, created after following the official guide.

## 1. Introduction

Official page: [PyBullet](https://pybullet.org/wordpress/)
Guide: [Quickstart](https://docs.google.com/document/d/10sXEhzFRSnvFcl3XxNGhnD4N2SedqwdAvK3dsihxVUA/)

Features:
- Robotics simulation and machine learning
- Robot models (URDF, etc.) can be loaded
- Forward & inverse kinematics + dynamics available
- Collision detection
- Visualization
- Shared memory, TCP, UDP communication

Installation:
```bash
# conda environment
source/conda activate 3d
# install
pip3 install pybullet --upgrade --user
python3 -m pybullet_envs.examples.enjoy_TF_AntBulletEnv_v0_2017may
python3 -m pybullet_envs.examples.enjoy_TF_HumanoidFlagrunHarderBulletEnv_v1_2017jul
python3 -m pybullet_envs.deep_mimic.testrl --arg_file run_humanoid3d_backflip_args.txt
```

### 1.1 Hello World

In [3]:
# PyBullet
import pybullet as p
import time
# Sample loader: off-the-shelf samples accessible
import pybullet_data

PyBullet has a Client-Server Architecture: The Physics is the server that serves data requested by a client. Several servers or connection modes are available; the two most common ones are:
- `DIRECT`: only simulation, no graphics nor GUI displayed, but rendered images can be obtained
- `GUI`: simulation + graphics (with GUI)

The graphics run in a separate thread for `GUI` on Windows/Linux, in the same for OSX.

The other connection modes refer to cases where we connect to a server in another process or even on another machine and require additional arguments (such as `hostName` and `port`):
- `SHARED_MEMORY`
- `UDP`
- `TCP_GUI_SERVER`
- `SHARED_MEMORY_SERVER`
- `SHARED_MEMORY_GUI`


In [4]:
# Select client connection mode to physics server: DIRECT, GUI, ...
# On Mac, if GUI selected, the OpenGL visualization is in the same thread...
#physicsClient = p.connect(p.GUI)
physicsClient = p.connect(p.DIRECT)
# Set search path for off-the-shelf samples or set own path with models
p.setAdditionalSearchPath(pybullet_data.getDataPath())
# physicsClientId is optional, for when we connect to several servers
p.setGravity(0,0,-10)
# Loaded from the path added above
# For each object we get an Id handle
# We can add several arguments:
# basePosition, baseOrientation, ...
# IMPORTANT parameter: useMaximalCoordinates:
# links are defined as 6DoF rigid bodies
planeId = p.loadURDF("plane.urdf")
startPos = [0,0,1]
startOrientation = p.getQuaternionFromEuler([0,0,0])
boxId = p.loadURDF("r2d2.urdf", startPos, startOrientation)
# Set the center of mass frame (loadURDF sets base link frame)
startPosOrn = p.resetBasePositionAndOrientation(boxId, startPos, startOrientation)
for i in range (100): # 10000
    p.stepSimulation()
    time.sleep(1./240.)
# Get pose of an object wit its id: Translation + Quaternion
cubePos, cubeOrn = p.getBasePositionAndOrientation(boxId)
print(cubePos,cubeOrn)

(1.5117627740528593e-08, 1.4889593646656074e-06, 0.47099230897457317) (-3.379035448734749e-08, 3.997113849372463e-09, 9.265643796039036e-08, 0.9999999999999951)


In [5]:
# Disconnect client from server
p.disconnect()

In [6]:
# Check where the sample models are located
pybullet_data.getDataPath()

'/Users/mxagar/.local/lib/python3.8/site-packages/pybullet_data'

In [7]:
# Object ID
boxId

1

In [8]:
# Physics client
physicsClient

0

In [9]:
# Convert Quaternion -> Rotation matrix (row-major)
p.getMatrixFromQuaternion(cubeOrn)

(0.9999999999999828,
 -1.853128761909076e-07,
 7.994221436957118e-09,
 1.8531287565065205e-07,
 0.9999999999999806,
 6.758070971541131e-08,
 -7.994233960532654e-09,
 -6.758070823397798e-08,
 0.9999999999999977)

In [10]:
# Get information of the physics client
p.getConnectionInfo(physicsClient)

{'isConnected': 0, 'connectionMethod': 0}

#### Connecting several clients

We can start several parallel simulations with `pybullet_utils.bullet_client`.
Examples in:
- [env_bases.py](https://github.com/bulletphysics/bullet3/blob/master/examples/pybullet/gym/pybullet_envs/env_bases.py)
- [multipleScenes.py](
https://github.com/bulletphysics/bullet3/blob/master/examples/pybullet/gym/pybullet_utils/examples/multipleScenes.py)

In [11]:
from pybullet_utils import bullet_client

In [12]:
c = bullet_client.BulletClient(p.GUI)

In [14]:
c.disconnect()

#### URDF files

We define robot systems with them.

[URDF Tutorials](http://wiki.ros.org/urdf/Tutorials)